In [37]:
import os
import pandas as pd
from PIL import Image

# Paths
path2train = "data/train"
excel_path = os.path.join(path2train, "image_labels.xlsx")

# Function to create labels DataFrame and save to Excel
def create_labels_dataframe(path2train, excel_path):
    data = []
    index = 0
    for img_name in os.listdir(path2train):
        if img_name.endswith('.jpg'):
            img_path = os.path.join(path2train, img_name)
            img = Image.open(img_path)
            img = img.resize((224, 224))  # Resize image to (224, 224)
            img.save(img_path)  # Save resized image, overwriting the original
            
            label = 0 if 'cat' in img_name else 1  # Assign label: 0 for cats, 1 for dogs
            data.append((index, img_name, label))
            index += 1
    
    labels_df = pd.DataFrame(data, columns=['index', 'imgName', 'label'])
    labels_df.to_excel(excel_path, index=False)  # Save to Excel file
    return labels_df

# Create labels dataframe
labels_df = create_labels_dataframe(path2train, excel_path)


In [38]:
import torchvision.transforms as transforms

# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
])

# Only resizing and normalization for validation
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Update Dataset class to accept different transforms
class AMD_dataset(Dataset):
    def __init__(self, path2data, transform=None):
        self.path2labels = os.path.join(path2data, "image_labels.xlsx")
        self.labels_df = pd.read_excel(self.path2labels)
        self.labels = self.labels_df["label"].values
        self.img_names = self.labels_df["imgName"]
        self.ids = self.labels_df.index
        self.transform = transform
        self.path2train = path2data

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_name = self.img_names[idx]
        img_path = os.path.join(self.path2train, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Create dataset instances with the new transforms
train_ds = AMD_dataset(path2train, transform_train)
val_ds = AMD_dataset(path2train, transform_val)

# Create DataLoaders
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=32, shuffle=False)


In [39]:
from sklearn.model_selection import ShuffleSplit
from torch.utils.data import Subset, DataLoader

# Split the dataset into training and validation sets
sss = ShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
indices = range(len(amd_ds))
for train_index, val_index in sss.split(indices):
    print(len(train_index))
    print("-" * 10)
    print(len(val_index))

# Create training and validation subsets
train_ds = Subset(amd_ds, train_index)
val_ds = Subset(amd_ds, val_index)

print(len(train_ds))
print(len(val_ds))

# DataLoader
train_dl = DataLoader(train_ds, batch_size=5, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=3, shuffle=True)


445
----------
112
445
112


In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchsummary import summary

# CNN Model
class Net(nn.Module):
    def __init__(self, params):
        super(Net, self).__init__()
        C_in, H_in, W_in = params["input_shape"]
        init_f = params["initial_filters"]
        num_fc1 = params["num_fc1"]
        num_classes = params["num_classes"]
        self.dropout_rate = params["dropout_rate"]

        self.conv1 = nn.Conv2d(C_in, init_f, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(init_f)
        self.conv2 = nn.Conv2d(init_f, 2 * init_f, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(2 * init_f)
        self.conv3 = nn.Conv2d(2 * init_f, 4 * init_f, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(4 * init_f)
        self.conv4 = nn.Conv2d(4 * init_f, 8 * init_f, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(8 * init_f)

        h, w = self.findConv2dOutShape(H_in, W_in, self.conv1)
        h, w = self.findConv2dOutShape(h, w, self.conv2)
        h, w = self.findConv2dOutShape(h, w, self.conv3)
        h, w = self.findConv2dOutShape(h, w, self.conv4)

        self.num_flatten = h * w * 8 * init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def findConv2dOutShape(self, H_in, W_in, conv, pool=2):
        kernel_size = conv.kernel_size
        stride = conv.stride
        padding = conv.padding
        dilation = conv.dilation
        H_out = np.floor((H_in + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) / stride[0] + 1)
        W_out = np.floor((W_in + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) / stride[1] + 1)
        if pool:
            H_out /= pool
            W_out /= pool
        return int(H_out), int(W_out)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, self.num_flatten)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, self.dropout_rate, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Model parameters
params_model = {
    "input_shape": (3, 224, 224),
    "initial_filters": 8,
    "num_fc1": 100,
    "dropout_rate": 0.25,
    "num_classes": 2,
}

model = Net(params_model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(model)
print(next(model.parameters()).device)

summary(model, input_size=(3, 224, 224), device=device.type)




Net(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=12544, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=2, bias=True)
)
cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 224, 224]             224
       Batch

In [42]:
from torch.optim.lr_scheduler import StepLR

def train_model(model, train_dl, val_dl, device, num_epochs=20, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
    
    best_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_dl:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_train_loss = running_loss / len(train_dl)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}")
        
        model.eval()
        val_running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in val_dl:
                images, labels = images.to(device), labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        avg_val_loss = val_running_loss / len(val_dl)
        val_accuracy = 100 * correct / total
        print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pth')
    
    print("Training complete. Best Validation Accuracy: {:.2f}%".format(best_accuracy))

train_model(model, train_dl, val_dl, device)


Epoch [1/20], Train Loss: 1.5605
Validation Loss: 0.6777, Validation Accuracy: 54.46%
Epoch [2/20], Train Loss: 0.6633
Validation Loss: 0.6651, Validation Accuracy: 62.50%
Epoch [3/20], Train Loss: 0.6430
Validation Loss: 0.7011, Validation Accuracy: 58.93%
Epoch [4/20], Train Loss: 0.6305
Validation Loss: 0.6618, Validation Accuracy: 61.61%
Epoch [5/20], Train Loss: 0.5986
Validation Loss: 0.6355, Validation Accuracy: 64.29%
Epoch [6/20], Train Loss: 0.5500
Validation Loss: 0.7438, Validation Accuracy: 58.93%
Epoch [7/20], Train Loss: 0.5607
Validation Loss: 0.6361, Validation Accuracy: 60.71%
Epoch [8/20], Train Loss: 0.5060
Validation Loss: 0.5946, Validation Accuracy: 64.29%
Epoch [9/20], Train Loss: 0.5062
Validation Loss: 0.6443, Validation Accuracy: 65.18%
Epoch [10/20], Train Loss: 0.4543
Validation Loss: 0.6133, Validation Accuracy: 66.07%
Epoch [11/20], Train Loss: 0.4344
Validation Loss: 0.6992, Validation Accuracy: 63.39%
Epoch [12/20], Train Loss: 0.4523
Validation Loss: 0

In [43]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Define the model class
class Net(nn.Module):
    def __init__(self, params):
        super(Net, self).__init__()
        C_in, H_in, W_in = params["input_shape"]
        init_f = params["initial_filters"]
        num_fc1 = params["num_fc1"]
        num_classes = params["num_classes"]
        self.dropout_rate = params["dropout_rate"]

        self.conv1 = nn.Conv2d(C_in, init_f, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(init_f)
        self.conv2 = nn.Conv2d(init_f, 2 * init_f, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(2 * init_f)
        self.conv3 = nn.Conv2d(2 * init_f, 4 * init_f, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(4 * init_f)
        self.conv4 = nn.Conv2d(4 * init_f, 8 * init_f, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(8 * init_f)

        h, w = 224 // 16, 224 // 16  # 224 is input size and we divide by 2^4 because of 4 pooling layers
        self.num_flatten = h * w * 8 * init_f

        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, self.num_flatten)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, self.dropout_rate, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Define the image transformation
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Define the test dataset class
class TestDataset(Dataset):
    def __init__(self, path2data, transform=None):
        self.img_names = [f for f in os.listdir(path2data) if f.endswith('.jpg')]
        self.path2data = path2data
        self.transform = transform

    def __len__(self):
        return len(self.img_names)
    
    def __getitem__(self, idx):
        img_name = self.img_names[idx]
        img_path = os.path.join(self.path2data, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_name

# Create test dataset and dataloader
path2test = "data/test"
test_dataset = TestDataset(path2test, transform_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model parameters
params_model = {
    "input_shape": (3, 224, 224),
    "initial_filters": 8,
    "num_fc1": 100,
    "dropout_rate": 0.25,
    "num_classes": 2,
}

# Initialize and load the trained model
model = Net(params_model)
model.load_state_dict(torch.load('best_model.pth'))
model.eval()  # Set the model to evaluation mode

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Perform inference on the test images
results = []
with torch.no_grad():
    for images, img_names in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predicted = predicted.cpu().numpy()
        results.extend(zip(img_names, predicted))

# Map predictions to labels
label_map = {0: 'cat', 1: 'dog'}
predictions = [(img_name, label_map[pred]) for img_name, pred in results]

# Print predictions
for img_name, label in predictions:
    print(f"{img_name}: {label}")

# Save predictions to a CSV file
predictions_df = pd.DataFrame(predictions, columns=['Image', 'Label'])
predictions_df.to_csv('test_predictions.csv', index=False)


/var/folders/4w/t_rs_h2144q0rmw1kx3__qsc0000gn/T/ipykernel_44954/4017942060.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model

cat_190.jpg: cat
dog_147.jpg: dog
cat_147.jpg: cat
dog_219.jpg: dog
cat_542.jpg: cat
cat_595.jpg: dog
dog_191.jpg: dog
cat_422.jpg: dog
dog_344.jpg: dog
dog_150.jpg: dog
cat_583.jpg: cat
dog_227.jpg: cat
dog_421.jpg: cat
cat_384.jpg: cat
dog_380.jpg: dog
dog_155.jpg: dog
dog_141.jpg: dog
dog_196.jpg: dog
dog_551.jpg: cat
dog_237.jpg: cat
cat_586.jpg: dog
cat_545.jpg: dog
cat_223.jpg: dog
cat_551.jpg: cat
cat_587.jpg: dog
dog_236.jpg: dog
cat_140.jpg: dog
dog_197.jpg: cat
dog_168.jpg: dog
cat_342.jpg: dog
dog_28.jpg: cat
cat_430.jpg: cat
cat_418.jpg: cat
cat_395.jpg: cat
dog_354.jpg: dog
dog_142.jpg: dog
dog_181.jpg: dog
cat_156.jpg: cat
cat_585.jpg: cat
cat_234.jpg: cat
dog_194.jpg: dog
cat_355.jpg: dog
cat_433.jpg: cat
cat_341.jpg: cat
dog_369.jpg: cat
dog_355.jpg: dog
cat_332.jpg: cat
cat_468.jpg: cat
dog_124.jpg: dog
dog_130.jpg: dog
cat_124.jpg: dog
cat_118.jpg: dog
dog_534.jpg: dog
dog_520.jpg: dog
cat_520.jpg: cat
dog_521.jpg: cat
cat_290.jpg: dog
cat_119.jpg: dog
cat_88.jpg: cat